# Tutorial: Extracting Information from Text Using OpenAI's API

This tutorial demonstrates how to use OpenAI's API to extract information from a text input. We will go through the steps of setting up the OpenAI client, making requests to the API, and processing the responses. The example involves summarizing a Danish news article and extracting structured information from it.

## Installation and Imports
First, we need to install the required library and import necessary modules.


In [ ]:
!pip install openai -q

In [ ]:
# Import required libraries
from openai import OpenAI
from google.colab import userdata
import json
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
import textwrap

Setting Up the OpenAI Client

We will set up the OpenAI client using a custom API key and base URL.

In [ ]:
# Setup OpenAI client with custom API key and base URL
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

### Summarizing Text

We will call the language model to summarize a given Danish text into a single sentence.

In [ ]:
# Create client
client = OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY
)

In [ ]:
text = """
An ‘AI Scientist’ Is Inventing and Running Its Own Experiments
Letting programs learn through “open-ended” experimentation may unlock remarkable new capabilities, as well as new risks.

By Will Knight Aug 21, 2024 12:00 PM

At first glance, a recent batch of research papers produced by a prominent artificial intelligence lab at the University of British Columbia in Vancouver might not seem that notable. Featuring incremental improvements on existing algorithms and ideas, they read like the contents of a middling AI conference or journal.

But the research is, in fact, remarkable. That’s because it’s entirely the work of an “AI scientist” developed at the UBC lab together with researchers from the University of Oxford and a startup called Sakana AI.

The project demonstrates an early step toward what might prove a revolutionary trick: letting AI learn by inventing and exploring novel ideas. They’re just not super novel at the moment. Several papers describe tweaks for improving an image-generating technique known as diffusion modeling; another outlines an approach for speeding up learning in deep neural networks.

“These are not breakthrough ideas. They’re not wildly creative,” admits Jeff Clune, the professor who leads the UBC lab. “But they seem like pretty cool ideas that somebody might try.”

As amazing as today’s AI programs can be, they are limited by their need to consume human-generated training data. If AI programs can instead learn in an open-ended fashion, by experimenting and exploring “interesting” ideas, they might unlock capabilities that extend beyond anything humans have shown them.

Clune’s lab had previously developed AI programs designed to learn in this way. For example, one program called Omni tried to generate the behavior of virtual characters in several video-game-like environments, filing away the ones that seemed interesting and then iterating on them with new designs. These programs had previously required hand-coded instructions in order to define interestingness. Large language models, however, provide a way to let these programs identify what’s most intriguing because of their ability to mimic human reasoning. Another recent project from Clune’s lab used this approach to let AI programs dream up the code that allows virtual characters to do all sorts of things within a Roblox-like world.

The AI scientist is one example of Clune’s lab riffing on the possibilities. The program comes up with machine learning experiments, decides what seems most promising with the help of an LLM, then writes and runs the necessary code—rinse and repeat. Despite the underwhelming results, Clune says open-ended learning programs, as with language models themselves, could become much more capable as the computer power feeding them is ramped up.

“It feels like exploring a new continent or a new planet,” Clune says of the possibilities unlocked by LLMs. “We don't know what we're going to discover, but everywhere we turn, there's something new.”

Tom Hope, an assistant professor at the Hebrew University of Jerusalem and a research scientist at the Allen Institute for AI (AI2), says the AI scientist, like LLMs, appears to be highly derivative and cannot be considered reliable. “None of the components are trustworthy right now,” he says.

Hope points out that efforts to automate elements of scientific discovery stretch back decades to the work of AI pioneers Allen Newell and Herbert Simon in the 1970s, and, later, the work of Pat Langley at the Institute for the Study of Learning and Expertise. He also notes that several other research groups, including a team at AI2, have recently harnessed LLMs to help with generating hypotheses, writing papers, and reviewing research. “They captured the zeitgeist,” Hope says of the UBC team. “The direction is, of course, incredibly valuable, potentially.”

Whether LLM-based systems can ever come up with truly novel or breakthrough ideas also remains unclear. “That’s the trillion-dollar question,” Clune says.

Even without scientific breakthroughs, open-ended learning may be vital to developing more capable and useful AI systems in the here and now. A report posted this month by Air Street Capital, an investment firm, highlights the potential of Clune’s work to develop more powerful and reliable AI agents, or programs that autonomously perform useful tasks on computers. The big AI companies all seem to view agents as the next big thing.

This week, Clune’s lab revealed its latest open-ended learning project: an AI program that invents and builds AI agents. The AI-designed agents outperform human-designed agents in some tasks, such as math and reading comprehension. The next step will be devising ways to prevent such a system from generating agents that misbehave. “It's potentially dangerous,” Clune says of this work. “We need to get it right, but I think it's possible.”
"""

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    #model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",

    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Summarize following in 1 sentence: " + text ,
        },
    ],
)

output = chat_completion.choices[0].message.content

In [ ]:
print(textwrap.fill(output, width=80))

## Creating a User Object and extracting structured info
We will define a schema for a user and call the API to create a user object based on this schema.

In [ ]:
# Define the schema for the output.
class User(BaseModel):
    name: str = Field(description="user name")
    address: str = Field(description="address")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    response_format={"type": "json_object", "schema": User.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Create a user named Alice, who lives in 42, Wonderland Avenue.",
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))

## Extracting Case Details
We will define a schema for case details and extract relevant information from the given text.

In [ ]:
class ExtractScheme(BaseModel):
    title: str = Field(description="Title of the news article")
    publication_date: str = Field(description="Date when the article was published")

    main_topic: str = Field(description="Primary topic or focus of the article")

    research_description: str = Field(description="Brief description of the AI research or development")
    institutions_involved: List[str] = Field(description="Organizations or institutions mentioned in the research")
    key_researchers: List[str] = Field(description="Main researchers or scientists mentioned")

    ai_technologies: List[str] = Field(description="AI technologies or methods discussed")
    research_outcomes: List[str] = Field(description="Key results or findings from the research")

    potential_impacts: List[str] = Field(description="Potential significance or impacts of the research")
    ethical_concerns: Optional[List[str]] = Field(description="Ethical or societal concerns related to the research")

    keywords: List[str] = Field(description="Key terms or phrases from the article")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    response_format={"type": "json_object", "schema": ExtractScheme.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are an AI model tasked with extracting structured information from a text related to AI research and technology. Follow the schema provided below to extract the relevant details. You do not invent information that is not in the provided text.",
        },
        {
            "role": "user",
            "content": "Extract case informatin form following.: " + text,
        },
    ],
)



In [ ]:
extracted_output = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(extracted_output, ensure_ascii=False, indent=2))

In [ ]:
ExtractScheme.model_json_schema()

In [ ]:
json_schema = str(ExtractScheme.model_json_schema())

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are an AI model tasked with extracting structured information from a text related to AI research and technology. Follow the schema provided below to extract the relevant details. You do not invent information that is not in the provided text. You output JSON only. Nothing else.",
        },
        {
            "role": "user",
            "content": "Extract case informatin form following.: " + text + "Use following JSON schema:" + json_schema,
        },
    ],
)



In [ ]:
extracted_output = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(extracted_output, ensure_ascii=False, indent=2))


In [ ]:
text_2 = """

Anthropic launches Claude Artifacts generally for all users, mobile
Carl Franzen

Join our daily and weekly newsletters for the latest updates and exclusive content on industry-leading AI coverage. Learn More

Earlier this summer, the San Francisco-based AI startup Anthropic — a leading rival of OpenAI when it comes to developing useful new large language models (LLMs) — unveiled a surprise new feature it called “Artifacts.”

The feature allowed users of Anthropic’s Claude family of LLMs and chatbots on the web to enable a new window that would appear alongside their chat interface and run code snippets and even full programs generated by the LLM at the user’s request.

For example, a user could ask Claude to generate a simple interactive visualization, chart, or a playable game and run it alongside the chatbot right in their browser. The feature was impressive enough that VentureBeat editorial director Michael Nuñez called it “this year’s most important AI feature” and users have since generated tens of millions of Artifacts since its release, according to Anthropic.

However, users previously had to turn on Claude Artifacts manually by clicking their username initials in the lower left corner of the Claude chatbot screen, select “Feature Preview” and toggle on Artifacts. But no more: Anthropic today announced the general availability of Artifacts across its Free, Pro, and Team tiers, as well as its available on the official Claude iOS and Android mobile apps, making it easier to create and interact with interactive code on the go.

Anthropic’s Head of Developer Relations Alex Albert posted on the social network X that he spent “all morning replicating simple games with Claude. We’re nearing the era of mobile apps created in real-time by LLMs.”

We launched Artifacts on iOS and Android today!

I've spent all morning replicating simple games with Claude.

We nearing the era of mobile apps created in real-time by LLMs. pic.twitter.com/ONSVzq9FKy
— Alex Albert (@alexalbert__) August 27, 2024
Only Free and Pro plans will have the ability to publish and remix Artifacts with the broader Claude community. This feature allows users to build upon and iterate on content created by others worldwide, facilitating a dynamic exchange of ideas and resources.

For users on the Team plan, Artifacts can be shared within Projects, enabling secure and efficient collaboration among team members.

Anthropic expects Artifacts to streamline workflows and enhance productivity by allowing teams to use it collaboratively and iterate on one another’s creations securely over the web.

The strategy behind Claude Artifacts: user experience over raw power?

While much of the AI development world has focused on enhancing raw processing power and expanding model capabilities, Artifacts represents a focus on user experience and redesigning AI interfaces away from the simple chatbot model.

In a way, I believe it is analogous to Nintendo in gaming — the company often comes out with game consoles that have much less processing and graphics power than rivals at Microsoft and Sony, but seeks to gain users with novel user interfaces, often to great success.

Anthropic envisions Artifacts as a versatile tool that can be utilized by teams across various industries to accelerate the creation of high-quality work products.

The platform supports a wide range of outputs, including code snippets, flowcharts, SVG graphics, websites, and interactive dashboards.

For instance, developers can now create architecture diagrams directly from their codebases, product managers can develop interactive prototypes for rapid feature testing, and designers can produce visualizations for quick prototyping. Similarly, marketers can design campaign dashboards rich with performance metrics, while sales teams can visualize their pipelines and forecast insights more effectively.

As Artifacts become a standard part of the Claude experience, Anthropic anticipates seeing a wide array of creative and practical uses emerge from its global user base.

VB Daily

Stay in the know! Get the latest news in your inbox daily

By subscribing, you agree to VentureBeat's Terms of Service.
"""

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are an AI model tasked with extracting structured information from a text related to AI research and technology. Follow the schema provided below to extract the relevant details. You do not invent information that is not in the provided text. You output JSON only. Nothing else.",
        },
        {
            "role": "user",
            "content": "Extract case informatin form following.: " + text + "Use following JSON schema:" + json_schema,
        },
                {
            "role": "assistant",
            "content": """{
  "title": "AI Scientist",
  "publication_date": "Aug 21, 2024",
  "author": "Will Knight",
  "main_topic": "AI research",
  "research_description": "A research project that demonstrates an early step toward letting AI learn by inventing and exploring novel ideas.",
  "institutions_involved": [
    "University of British Columbia (UBC)",
    "University of Oxford",
    "Sakana AI"
  ],
  "key_researchers": [
    "Jeff Clune (professor and lead of the UBC lab)",
    "Tom Hope (assistant professor at the Hebrew University of Jerusalem and a research scientist at AI2)"
  ],
  "ai_technologies": [
    "Large language models (LLMs)",
    "Diffusion modeling",
    "Deep neural networks",
    "Open-ended learning",
    "AI agents"
  ],
  "research_outcomes": [
    "Incremental improvements on existing algorithms and ideas",
    "Tweaks for improving an image-generating technique",
    "Approach for speeding up learning in deep neural networks",
    "AI program that invents and builds AI agents"
  ],
  "potential_impacts": [
    "Unlocking remarkable new capabilities",
    "Introducing new risks",
    "Developing more capable and useful AI systems",
    "Creating AI agents that autonomously perform useful tasks"
  ],
  "ethical_concerns": [
    "Potential dangers of generating misbehaving AI agents"
  ],
  "keywords": [
    "AI scientist",
    "open-ended learning",
    "large language models",
    "AI agents",
    "diffusion modeling",
    "deep neural networks"
  ]
}""",
        },
                {
            "role": "user",
            "content": "Extract case informatin form following.: " + text_2 + "Use following JSON schema:" + json_schema,
        },
    ],
)



In [ ]:
chat_completion.choices[0].message.content

In [ ]:
extracted_output = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(extracted_output, ensure_ascii=False, indent=2))
